In [1]:
#Import all necessary libraries
import numpy as np
import pandas as pd
import glob
import os
import csv
from pymongo import MongoClient
import json
import math
import re

In [49]:
#Convert to excel for analysis of data
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06'
#Find all csv files in this path
csv_files = [os.path.basename(_).split(".")[0] for _ in glob.glob(os.path.join(path, '*.csv'))]

#Read all files and save as excel
for f in csv_files:
    read_file = pd.read_csv(path+'\\'+f+".csv")
    read_file.to_excel(path+'\\'+f+'.xlsx', index = None, header=True)       

c:\Users\damia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\damia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
#Check how many columns are empty
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.xlsx'))]

print(csv_files)
for f in csv_files:
    print('\n')
    print(f)
    dataempty = pd.read_excel(path+'\\'+f+'.xlsx')
    print('Total number of rows : {0}'.format(len(dataempty.index)))
    print(dataempty.isnull().sum())
    print('\n')
    

['forward', 'horses_1990', 'horses_1991', 'horses_1992', 'horses_1993', 'horses_1994', 'horses_1995', 'horses_1996', 'horses_1997', 'horses_1998', 'horses_1999', 'horses_2000', 'horses_2001', 'horses_2002', 'horses_2003', 'horses_2004', 'horses_2005', 'horses_2006', 'horses_2007', 'horses_2008', 'horses_2009', 'horses_2010', 'horses_2011', 'horses_2012', 'horses_2013', 'horses_2014', 'horses_2015', 'horses_2016', 'horses_2017', 'horses_2018', 'horses_2019', 'horses_2020', 'races_1990', 'races_1991', 'races_1992', 'races_1993', 'races_1994', 'races_1995', 'races_1996', 'races_1997', 'races_1998', 'races_1999', 'races_2000', 'races_2001', 'races_2002', 'races_2003', 'races_2004', 'races_2005', 'races_2006', 'races_2007', 'races_2008', 'races_2009', 'races_2010', 'races_2011', 'races_2012', 'races_2013', 'races_2014', 'races_2015', 'races_2016', 'races_2017', 'races_2018', 'races_2019', 'races_2020']


forward
Total number of rows : 33708
course             11
countryCode         0
market

In [10]:
#Data cleaning

#Function for convert string distance to 0.1, as the length for different abberviations for distance in for example horse neck, body is very small.
#A simplification was made to convert it to 0.1
def convertdistances(distance_text):
    #if (str(distance_text).isalpha() and type(distance_text) == str) or (distance_text == "sht-hd" or distance_text == "4min"):
    if re.search('[a-zA-Z]', str(distance_text)):
        return 0.1
    return distance_text
#Read all files in this folder
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06\\Horses'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.xlsx'))]

for f in csv_files:
    horse = pd.read_excel(path+'\\'+f+'.xlsx')
    #For all files, drop columns "weightSt","weightLb","overWeight","outHandicap","headGear.
    horse = horse.drop(["weightSt","weightLb","overWeight","outHandicap","headGear"], axis= 1)
    # If price is one of the columns, drop it. Only 2020 dataset has price.
    if "price" in horse.columns:
        horse = horse.drop(["price"], axis=1)
    
    #Convert positionL and dist through the function
    horse["positionL"] = horse["positionL"].map(convertdistances)
    horse["dist"] = horse["dist"].map(convertdistances)    
      
    #Save as csv so mongodb can read it
    horse.to_csv(path+'\\'+f+'cleaned.csv', index = None, header=True)    

In [12]:
#Import horses to mongodb
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06\\Horses'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.csv'))]

def mongoimport(csv_path, db_name, coll_name, db_url='localhost', db_port=27017):
    print(csv_path)
    types = {
        'rid': "int",
        'age': "int",
        'saddle': "int",
        'decimalPrice': 'float',
        'isFav': 'int',
        'position': 'int',
        'positionL': 'float',
        'dist': 'float',
        'RPR': 'int',
        'TR': 'int',
        'OR': 'int',
        'runners': 'int',
        'margin': 'float',
        'weight': 'int',
        'res_win': 'int',
        'res_place': 'int'
    }
    url = f"mongodb://{db_url}:{db_port}"
    client = MongoClient(url)
    db = client[db_name]
    coll = db[coll_name]    
    csv_file = open(csv_path, 'r')
    reader = csv.DictReader(csv_file)    
    for idx,row in enumerate(reader):
        for key, val in row.items():
            if key in types and val != "":
                if types[key] == "int":
                    try:
                        row[key] = int(math.floor(float(val)))
                    except:
                        print(key, val, idx+1)
                        return 0
                else:
                    try:
                        row[key] = float(val)
                    except:
                        print(key, val, idx+1)
                        return 0
        coll.insert_one(row)
    return 1

for idx, f in enumerate(csv_files):
    #horse = pd.read_csv(path+'\\'+f+'.csv')
    print(f"Inserting {idx+1} out of {len(csv_files)}")
    x = mongoimport(csv_path=path+'\\'+f+'.csv', db_name="HorsesPY", coll_name="Horse") 
    if x == 0:
        break
    
    

Inserting 1 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1990cleaned.csv
Inserting 2 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1991cleaned.csv
Inserting 3 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1992cleaned.csv
Inserting 4 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1993cleaned.csv
Inserting 5 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1994cleaned.csv
Inserting 6 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1995cleaned.csv
Inserting 7 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1996cleaned.csv
Inserting 8 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1997cleaned.csv
Inserting 9 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1998cleaned.csv
Inserting 10 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_1999cleaned.csv
Inserting 11 out of 31
c:\Users\damia\Desktop\Betting\DVGB06\Horses\horses_2000cleaned.csv
Insertin

In [4]:
#Import data to mongodb manually - Save all files as one bigger file
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06\\Horses'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.csv'))]

#Create array and append all files to one array
li = []
for f in csv_files:
    horse = pd.read_csv(path+'\\'+f+'.csv', index_col=None, header=0)
    li.append(horse)
#Create a new dataframe with values of all files and save as csv
frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv(path+'\\Horse.csv', index = None, header=True)    



c:\Users\damia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\damia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\damia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#Clean races

#Delete parentheses in chosen column
def parenthesesdeleter(text):
    if "(" in text:
        tmp = text.index("(")
        return text[0:tmp]
    

#Clean date data
def cleandate(string_text):
    if ' ' in string_text:
        date, empty = string_text.split(" ", 1)
        return date
    return string_text
#Convert distance from furlong/miles to meters
def distanceconverter(string_text):
    sum = 0.0
    m_ends = 0
    string_distance = []
    #Replace 1/2 with .5
    string_distance = string_text.replace("½",".5")
    for x in range(len(string_distance)):
        #Find if meters then convert to KM
        if string_distance[x] == "m":
            sum += float(string_distance[0:x]) *  1.609344
            m_ends = x+1
        #Find if furlong then convert to km
        if string_distance[x] == "f":
            sum += float(string_distance[m_ends:x]) * 0.201168
    return sum

#If hurdles are in the string, return their digit
def converttohurdles(string_text):
    if "hurdles" in str(string_text):
        return "".join(filter(str.isdigit, string_text))        

#If fences are in the string, return their digit
def converttofences(string_text):
    if "fences" in str(string_text):
        return "".join(filter(str.isdigit, string_text))     

path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06\\Races'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.xlsx'))]

for f in csv_files:
    race = pd.read_excel(path+'\\'+f+'.xlsx')
    if "currency" in race.columns:
        race = race.drop(["currency"], axis=1)
    #Delete parenthese function
    race["course"] = race["course"].map(parenthesesdeleter)
    #Convert miles and furlong to KM
    race["distance"] = race["distance"].map(distanceconverter)
    #Convert to hurdles and fences
    race["fences"] = race["hurdles"].map(converttofences)
    race["hurdles"] = race["hurdles"].map(converttohurdles)
    race["date"] = race["date"].map(cleandate)
    #Split date into years,months,days
    race[["Year","Month","Day"]] = race["date"].str.split("/", expand=True)
    #Drop rclass, date, ncond, title
    race[['Hours', 'Minutes']] = race["time"].str.split(":", expand=True)
    race = race.drop(["prizes", "date", "condition", "title","ncond"], axis= 1)
    race.to_csv(path+'\\'+f+'cleaned.csv', index = None, header=True)    

In [10]:
#Import races to mongodb
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06\\Races'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.csv'))]

def mongoimport(csv_path, db_name, coll_name, db_url='localhost', db_port=27017):
    #Set columns to correct data type
    #Dictionary that defines the data type corresponding to a column value
    types = {
        'rid': "int",
        'distance': "float",
        'winningTime': "float",
        'prize': 'int',
        'metric': 'float',
        'class': 'int',
        'hurdles': 'int',
        'Year': 'int',
        'Month': 'int',
        'Day': 'int',
        'Hours': 'int',
        'Minutes': 'int',
        'fences': 'int',
    }
    #Connect to collection via pymongo via connectionstring
    #Since db is local connectionstring is just localhost
    #Will put in environment variable if posted to github
    url = f"mongodb://{db_url}:{db_port}"
    client = MongoClient(url)
    db = client[db_name]
    coll = db[coll_name]    


    #Opens csv file
    csv_file = open(csv_path, 'r')
    reader = csv.DictReader(csv_file)    
    #Iterate through every row in the csv file, then change the cell value to correct type
    #Lastly insert document into db
    for idx,row in enumerate(reader):
        for key, val in row.items():
            if key in types and val != "":
                if types[key] == "int":
                    row[key] = int(math.floor(float(val)))
                else:
                    row[key] = float(val)
        coll.insert_one(row)    

#Iterate over csv files found in folder
for idx, f in enumerate(csv_files):
    #horse = pd.read_csv(path+'\\'+f+'.csv')
    print(f"Inserting {idx+1} out of {len(csv_files)}")
    mongoimport(csv_path=path+'\\'+f+'.csv', db_name="HorsesPY", coll_name="Races") 
    

Inserting 1 out of 31
Inserting 2 out of 31
Inserting 3 out of 31
Inserting 4 out of 31
Inserting 5 out of 31
Inserting 6 out of 31
Inserting 7 out of 31
Inserting 8 out of 31
Inserting 9 out of 31
Inserting 10 out of 31
Inserting 11 out of 31
Inserting 12 out of 31
Inserting 13 out of 31
Inserting 14 out of 31
Inserting 15 out of 31
Inserting 16 out of 31
Inserting 17 out of 31
Inserting 18 out of 31
Inserting 19 out of 31
Inserting 20 out of 31
Inserting 21 out of 31
Inserting 22 out of 31
Inserting 23 out of 31
Inserting 24 out of 31
Inserting 25 out of 31
Inserting 26 out of 31
Inserting 27 out of 31
Inserting 28 out of 31
Inserting 29 out of 31
Inserting 30 out of 31
Inserting 31 out of 31


In [5]:
#Import data to mongodb - Manually 
path = 'c:\\Users\\damia\\Desktop\\Betting\\DVGB06\\Races'
csv_files = [os.path.basename(_).split('.')[0] for _ in glob.glob(os.path.join(path, '*.csv'))]

#Create array and append all files to one array
li = []
for f in csv_files:
    races = pd.read_csv(path+'\\'+f+'.csv', index_col=None, header=0)
    li.append(races)

#Create a new dataframe with values of all files and save as csv
frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv(path+'\\Races.csv', index = None, header=True)   